In [2]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
from lightgbm import early_stopping


In [3]:
# Load dữ liệu
red = pd.read_csv("Red.csv")
white = pd.read_csv("White.csv")
rose = pd.read_csv("Rose.csv")
sparkling = pd.read_csv("Sparkling.csv")

# Gắn nhãn và gộp
red["WineStyle"] = "Red"
white["WineStyle"] = "White"
rose["WineStyle"] = "Rose"
sparkling["WineStyle"] = "Sparkling"
df = pd.concat([red, white, rose, sparkling], ignore_index=True)

# Xử lý dữ liệu cơ bản
df = df[df["Year"] != "N.V."].copy()
df["Year"] = df["Year"].astype(int)
df["Age"] = 2025 - df["Year"]
df["Variety"] = "Unknown"

# Đặc trưng cơ bản
df["PricePerAge"] = df["Price"] / (df["Age"] + 1)
df["IsOld"] = df["Age"] >= 10
df["LogPrice"] = np.log1p(df["Price"])
df["PriceAgeInteraction"] = df["PricePerAge"] * df["LogPrice"]
df["LogLogPrice"] = np.log1p(df["LogPrice"])

# SmoothedRating
global_mean = df["Rating"].mean()
df["SmoothedRating"] = (df["Rating"] * df["NumberOfRatings"] + global_mean * 5) / (df["NumberOfRatings"] + 5)

# Phân nhóm độ tin cậy
def reliability_group(n):
    if n < 35: return "Low"
    elif n < 838: return "Medium"
    else: return "High"
df["ReliabilityGroup"] = df["NumberOfRatings"].apply(reliability_group)

# Đặc trưng nâng cao: theo quốc gia, vùng, nhà máy, giống nho
df["Country_Rank"] = df["Country"].map(df.groupby("Country")["SmoothedRating"].mean().rank(ascending=False, method="min").astype(int))
df["Region_Rank"] = df["Region"].map(df.groupby("Region")["SmoothedRating"].mean().rank(ascending=False, method="min").astype(int))
winery_mean = df.groupby("Winery")["SmoothedRating"].mean()
df["Winery_Score"] = df["Winery"].map((winery_mean.max() - winery_mean) / (winery_mean.max() - 1))
df["Variety_Freq"] = df["Variety"].map(df["Variety"].value_counts(normalize=True))

# 🔍 Đặc trưng bổ sung khai thác NumberOfRatings
df["LogNumberOfRatings"] = np.log1p(df["NumberOfRatings"])
df["RatingSupportScore"] = df["NumberOfRatings"] / (df["Age"] + 1)


In [4]:

features = ["Country", "Region", "Winery", "Price", "WineStyle", "Variety", "Age",
            "NumberOfRatings", "PricePerAge", "IsOld", "LogPrice", "PriceAgeInteraction",
            "LogLogPrice", "Country_Rank", "Region_Rank", "Winery_Score", "Variety_Freq",
            "LogNumberOfRatings", "RatingSupportScore"]
target = "SmoothedRating"

# Encode object features
df_encoded = df.copy()
for col in df_encoded[features]:
    if df_encoded[col].dtype == "object":
        df_encoded[col] = df_encoded[col].astype("category")

low_df = df_encoded[df_encoded["ReliabilityGroup"] == "Low"].dropna(subset=features + [target])
mid_df = df_encoded[df_encoded["ReliabilityGroup"] == "Medium"].dropna(subset=features + [target])
high_df = df_encoded[df_encoded["ReliabilityGroup"] == "High"].dropna(subset=features + [target])

low_train, low_test = train_test_split(low_df, test_size=0.2, random_state=42)
mid_train, mid_test = train_test_split(mid_df, test_size=0.2, random_state=42)
high_train, high_test = train_test_split(high_df, test_size=0.2, random_state=42)

mid_sample_for_low = mid_train.sample(frac=0.5, random_state=1)
mid_sample_for_high = mid_train.sample(frac=0.5, random_state=2)
low_train_blended = pd.concat([low_train, mid_sample_for_low], ignore_index=True)
high_train_blended = pd.concat([high_train, mid_sample_for_high], ignore_index=True)


In [15]:
import lightgbm as lgb

def evaluate(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    return mae, rmse, r2, mape

# Train các mô hình và lưu kết quả
results = {}

for name, train_df, test_df in [
    ("Low", low_train_blended, low_test),
    ("Medium", mid_train, mid_test),
    ("High", high_train_blended, high_test)
]:
    X_train, y_train = train_df[features], train_df[target]
    X_test, y_test = test_df[features], test_df[target]

    # Tạo Dataset cho LightGBM
    train_set = lgb.Dataset(X_train, label=y_train)
    valid_set = lgb.Dataset(X_test, label=y_test, reference=train_set)

    params = {
        "objective": "regression",
        "metric": "rmse",
        "learning_rate": 0.03,
        "max_depth": 4,
        "min_child_samples": 10,
        "reg_alpha": 1.0,
        "reg_lambda": 2.0,
        "verbosity": -1
    }
    from lightgbm import early_stopping

    model = lgb.train(
        params,
        train_set,
        num_boost_round=1000,
        valid_sets=[valid_set],
        callbacks=[early_stopping(stopping_rounds=20)]
    )

    # Dự đoán
    y_train_pred = model.predict(X_train, num_iteration=model.best_iteration)
    y_test_pred = model.predict(X_test, num_iteration=model.best_iteration)


    results[name] = {
        "train": evaluate(y_train, y_train_pred),
        "test": evaluate(y_test, y_test_pred)
    }
    if name in ["Low", "Medium", "High"]:
        model.save_model(f"models/LightGBM_{name}.txt")




Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[257]	valid_0's rmse: 0.150784
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[709]	valid_0's rmse: 0.145618
Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[834]	valid_0's rmse: 0.119857


In [6]:
print("📊 Kết quả trên tập TEST:")
for group in ["Low", "Medium", "High"]:
    mae, rmse, r2, mape = results[group]["test"]
    print(f"{group:<7} Test  — MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")

print("\n🧪 Kết quả trên tập TRAIN:")
for group in ["Low", "Medium", "High"]:
    mae, rmse, r2, mape = results[group]["train"]
    print(f"{group:<7} Train — MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}, MAPE: {mape:.2f}%")


📊 Kết quả trên tập TEST:
Low     Test  — MAE: 0.1201, RMSE: 0.1508, R²: 0.6904, MAPE: 3.17%
Medium  Test  — MAE: 0.1123, RMSE: 0.1456, R²: 0.7262, MAPE: 2.93%
High    Test  — MAE: 0.0924, RMSE: 0.1199, R²: 0.8269, MAPE: 2.35%

🧪 Kết quả trên tập TRAIN:
Low     Train — MAE: 0.1052, RMSE: 0.1368, R²: 0.7621, MAPE: 2.76%
Medium  Train — MAE: 0.0938, RMSE: 0.1216, R²: 0.8142, MAPE: 2.46%
High    Train — MAE: 0.0888, RMSE: 0.1151, R²: 0.8385, MAPE: 2.32%


In [7]:
# 📊 Bảng so sánh chi tiết Train vs Test
import pandas as pd

df_eval = pd.DataFrame({
    "Group": ["Low", "Medium", "High"],
    "MAE_Train": [results["Low"]["train"][0], results["Medium"]["train"][0], results["High"]["train"][0]],
    "MAE_Test":  [results["Low"]["test"][0],  results["Medium"]["test"][0],  results["High"]["test"][0]],
    "R2_Train":  [results["Low"]["train"][2], results["Medium"]["train"][2], results["High"]["train"][2]],
    "R2_Test":   [results["Low"]["test"][2],  results["Medium"]["test"][2],  results["High"]["test"][2]]
})

df_eval["ΔMAE"] = df_eval["MAE_Test"] - df_eval["MAE_Train"]
df_eval["ΔR2"] = df_eval["R2_Test"] - df_eval["R2_Train"]

# Hiển thị bảng
from IPython.display import display
display(df_eval[["Group", "MAE_Train", "MAE_Test", "ΔMAE", "R2_Train", "R2_Test", "ΔR2"]])


Group  MAE_Train  MAE_Test      ΔMAE  R2_Train   R2_Test       ΔR2
0     Low   0.105231  0.120145  0.014914  0.762124  0.690381 -0.071743
1  Medium   0.093814  0.112260  0.018446  0.814158  0.726154 -0.088005
2    High   0.088784  0.092373  0.003589  0.838475  0.826938 -0.011537

In [14]:
print(os.listdir("models"))


['CatBoost_High.pkl', 'CatBoost_Low.pkl', 'CatBoost_Mid.pkl', 'LightGBM_High.txt', 'LightGBM_Low.txt', 'XGBoost_Encoders_High.pkl', 'XGBoost_Encoders_Low.pkl', 'XGBoost_Encoders_Mid.pkl', 'XGBoost_High.pkl', 'XGBoost_Low.pkl', 'XGBoost_Mid.pkl']


In [11]:
model.save_model("models/LightGBM_Low.txt")


In [10]:
import os
print(os.path.exists("models/LightGBM_Low.txt"))  # True nếu tồn tại



False


In [17]:
import lightgbm as lgb

model_low = lgb.Booster(model_file="models/LightGBM_Low.txt")
model_mid = lgb.Booster(model_file="models/LightGBM_Medium.txt")
model_high = lgb.Booster(model_file="models/LightGBM_High.txt")
